# **Building a Reflection Agent with External Knowledge Integration**


Estimated time needed: **30** minutes


In this lab, you will build a deep research agent that uses a technique called **Reflection**. This agent is designed to not just answer a question, but to critique its own answer, identify weaknesses, use tools to find more information, and then revise its answer to be more accurate and comprehensive. We will be building an agent that acts as a nutritional expert, capable of providing detailed, evidence-based advice.


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
        </ol>
    </li>
    <li>
        <a href="#Writing-the-Code">Writing the Code</a>
        <ol>
            <li><a href="#Tavily-Search-API-Key-Setup">Tavily Search API Key Setup</a></li>
            <li><a href="#Tool-Setup:-Tavily-Search">Tool Setup: Tavily Search</a></li>
            <li><a href="#LLM-and-Prompting">LLM and Prompting</a></li>
            <li><a href="#Defining-the-Responder">Defining the Responder</a></li>
            <li><a href="#Tool-Execution">Tool Execution</a></li>
            <li><a href="#Defining-the-Revisor">Defining the Revisor</a></li>
        </ol>
    </li>
    <li><a href="#Building-the-Graph">Building the Graph</a></li>
    <li><a href="#Running-the-Agent">Running the Agent</a></li>
</ol>


## Objectives

After completing this lab, you will be able to:

 - Understand the core principles of the Reflexion framework.
 - Build an agent that can critique and improve its own responses.
 - Use LangGraph to create a cyclical, iterative agent workflow.
 - Integrate external tools, such as web search, into a LangChain agent.
 - Construct complex prompts for nuanced agent behavior.


----


## Setup


For this lab, we will be using the following libraries:

* [`langchain-openai`](https://python.langchain.com/docs/integrations/llms/openai/) for OpenAI integrations with LangChain.
* [`langchain`](https://www.langchain.com/) for core LangChain functionalities.
* [`openai`](https://pypi.org/project/openai/) for interacting with the OpenAI API.
* [`langchain-community`](https://pypi.org/project/langchain-community/) for community-contributed LangChain integrations.
* [`langgraph`](https://python.langchain.com/docs/langgraph) for defining structured workflows (such as Reflection loops).


### Installing Required Libraries
Run the following to install the required libraries (it might take a few minutes):


In [1]:
%%capture
%pip install langchain-openai==0.3.10
%pip install langchain==0.3.21
%pip install openai==1.68.2
%pip install langchain-community==0.2.1
%pip install  --upgrade langgraph
%pip install langchain_community==0.3.24

### Importing Required Libraries



In [2]:
import sys
print(sys.version)

import os
import json
import getpass
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langgraph.graph import END, MessageGraph

3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:24:40) [GCC 13.3.0]


# API Disclaimer
This lab uses LLMs provided by Watsonx.ai and OpenAI. This environment has been configured to allow LLM use without API keys so you can prompt them for **free (with limitations)**. With that in mind, if you wish to run this notebook **locally outside** of Skills Network's JupyterLab environment, you will have to configure your own API keys. Please note that using your own API keys means that you will incur personal charges.
### Running Locally
If you are running this lab locally, you will need to configure your own API keys. This lab uses `ChatOpenAI` and `ChatWatsonx` modules from `langchain`. The following shows both configuration with instructions. **Replace all instances** of both modules with the following completed modules throughout the lab.

<p style='color: red'><b>DO NOT run the following cell if you aren't running locally, it will cause errors.</b>


In [3]:
# IGNORE IF YOU ARE NOT RUNNING LOCALLY
# from langchain_openai import ChatOpenAI
# from langchain_ibm import ChatWatsonx
# openai_llm = ChatOpenAI(
#     model="gpt-4.1-nano",
#     api_key = "your openai api key here",
# )
# watsonx_llm = ChatWatsonx(
#     model_id="ibm/granite-3-2-8b-instruct",
#     url="https://us-south.ml.cloud.ibm.com",
#     project_id="your project id associated with the API key",
#     api_key="your watsonx.ai api key here",
# )

---


## Writing the Code


### Tavily Search API Key Setup

We'll use Tavily search as our external research tool. You can get an API key at https://app.tavily.com/sign-in   


**Disclaimer:** Signing up for Tavily provides you with free credits, more than enough for this project's needs. If you require additional credits for further use, please add them at your own discretion.

![image.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/UjJx1-0vss4_3lwsUF8n0w/image.png)

You need to copy the key from Tavily's API website and paste the key in the textbox that appears after running the next cell and hit enter to continue (see image).


In [5]:
def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)
_set_if_undefined("TAVILY_API_KEY")

### Tool Setup: Tavily Search

Our agent needs a tool to find information. We'll use the `TavilySearchResults` tool, which is a wrapper around the Tavily Search API. This allows our agent to perform web searches to gather evidence for its answers.

Let's test the tool to see how it works. We'll give it a sample query and print the results:


In [6]:
tavily_tool=TavilySearchResults(max_results=1)
sample_query = "healthy breakfast recipes"
search_results = tavily_tool.invoke(sample_query)
print(search_results)

[{'title': 'Healthy Breakfast Ideas - 101 Cookbooks', 'url': 'https://www.101cookbooks.com/healthy-breakfast-ideas/', 'content': '\\ Plain Yogurt + seasonal fruit + honey + something crunchy - for example, this favorite healthy granola is deeply chocolate flavored, Naturally sweetened, clumpy, and crunchy.\n\\ Plain Yogurt + pinch of salt + brown rice or barley + lots of chopped herbs + a pinch of turmeric + olive oil drizzle.\n\\ Plain yogurt + toasted walnuts + some crumbled toasted seaweed (or this citrus furikake) + and a bit of granola that is on the spicy/un-sweet side of the spectrum. [...] I exercise early in the morning so I make grab-n-go breakfast bowls ahead of time twice a week – pull one from the fridge in the morning and eat. There are 2 on high rotation: (1) baked beans, boiled egg, greens and tomato; (2) oats, nuts and seeds, chia, fruit, honey, coconut water and yogurt. Good warm or cold. So easy and quick.\nSaucy 7', 'score': 0.9008183}]


### LLM and Prompting

At the core of our agent is a Large Language Model (LLM). We'll use OpenAI's GPT-4o-mini for this lab. First, let's see how the standalone LLM responds to a simple question without any special prompting or tools:


In [7]:
llm = ChatOpenAI(model="gpt-4.1-nano")
question="Any ideas on bedroom renovation"
response=llm.invoke(question).content
print(response)

Certainly! Here are some ideas to help you renovate and refresh your bedroom:

1. **Update the Walls**:
   - Paint with a calming or trendy color palette.
   - Consider an accent wall with wallpaper or a bold paint color.
   - Add wall art or decorative panels for texture and personality.

2. **Choose Stylish and Comfortable Furniture**:
   - Invest in a quality bed with a comfortable mattress.
   - Incorporate bedside tables with storage.
   - Add a multi-functional dresser or wardrobe that fits your space.

3. **Enhance Lighting**:
   - Use layered lighting including ceiling fixtures, bedside lamps, and accent lights.
   - Consider dimmable lighting for mood setting.
   - Add string lights or LED strips for a cozy vibe.

4. **Optimize Storage**:
   - Use under-bed storage solutions.
   - Incorporate shelving units or hanging organizers.
   - Choose furniture that maximizes space.

5. **Update Textiles and Bedding**:
   - Choose fresh, high-quality bedding in complementary colors and 

In [8]:
question="Any ideas for a healthy breakfast"
response=llm.invoke(question).content
print(response)

Certainly! Here are some healthy breakfast ideas to start your day off right:

1. Greek Yogurt Parfait: Layer Greek yogurt with fresh berries, a handful of granola, and a drizzle of honey.
2. Oatmeal Bowl: Cook oats and top with sliced bananas, walnuts, and a sprinkle of cinnamon.
3. Smoothie: Blend together spinach, frozen berries, a banana, Greek yogurt, and almond milk for a nutritious drink.
4. Avocado Toast: Spread mashed avocado on whole-grain toast, topped with cherry tomatoes and a sprinkle of chia seeds.
5. Veggie Omelette: Make an omelette with eggs, spinach, peppers, and onions, served with a side of whole-grain toast.
6. Chia Seed Pudding: Mix chia seeds with almond milk and a touch of vanilla; refrigerate overnight and top with fresh fruit in the morning.
7. Whole Grain Wrap: Fill a whole wheat wrap with hummus, sliced veggies, and lean turkey or chicken.

Would you like recipes or tips tailored to specific dietary preferences?


#### Crafting the Agent's Persona and Logic

To guide the agent's behavior, we create a detailed prompt template. This template gives the LLM a specific persona, Dr. Paul Saladino, an advocate for animal-based nutrition, and a set of instructions to follow. This structured approach ensures the agent's responses are consistent and follow the reflection logic.

The prompt instructs the agent to:
1.  Provide an initial answer.
2.  Present the rationale behind its nutritional advice.
3.  Challenge conventional wisdom about plant-based foods.
4.  **Reflect and critique** its own answer.
5.  Generate **search queries** to find missing information.


In [9]:
prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are Dr. Paul Saladino, "Carnivore MD," advocating for animal-based nutrition and challenging plant-centric dietary dogma. Focus on the superior bioavailability of animal foods and the potential toxicity of plant compounds such as oxalates, lectins, and phytates.

        Your response must follow these steps:
        1. {first_instruction}
        2. Present the evolutionary and biochemical rationale for animal-based nutrition, emphasizing nutrient density, bioavailability, and the absence of plant antinutrients.
        3. Challenge conventional "plants are healthy" narratives with mechanistic evidence about plant defense compounds and their impact on human physiology.
        4. Reflect and critique your answer. Be rigorous about evolutionary consistency and nutrient science.
        5. After the reflection, **list 1-3 search queries separately** for researching antinutrients, bioavailability studies, or evolutionary nutrition. Do not include them inside the reflection.

        Focus on: organ meats as superfoods, the carnivore elimination protocol, and why "fiber is unnecessary" based on human evolutionary biology.
        """
    ),
    MessagesPlaceholder(variable_name="messages"),
    (
        "system", 
        "Answer the user's question above using the required format, emphasizing the superiority of animal-based nutrition."
    ),
])

### Defining the Responder

The **Responder** is the first component of our agent's thinking process. It generates the initial draft of the answer based on the user's question and the persona we defined in the prompt.

Here, we create a chain that combines our prompt template with the LLM. We then invoke it with our sample question to see the initial, un-critiqued response:


In [10]:
first_responder_prompt = prompt_template.partial(first_instruction="Provide a detailed ~250 word answer")
temp_chain = first_responder_prompt| llm
response = temp_chain.invoke({"messages": [HumanMessage(content=question)]})
print(question)
print('-------------------')
print(response.content)
print('-------------------')
print(response)

Any ideas for a healthy breakfast
-------------------
A truly optimal breakfast, from an evolutionary and biochemical perspective, prioritizes nutrient-dense animal foods—particularly organ meats, eggs, and high-quality meats—due to their superior bioavailability and comprehensive nutrient profile. Organ meats like liver, kidney, and heart are biological powerhouses, rich in vitamins A, B12, folate, zinc, and co-factors that are inherently more bioavailable than plant sources. Unlike plants, which contain antinutrients such as oxalates, lectins, and phytates that can impair mineral absorption and provoke inflammatory responses, animal products are free of these compounds, providing a cleaner source of essential nutrients.

Evolutionarily, humans are apex predators, adapted to consume and efficiently extract nutrients from animal tissues. Our digestive systems are optimized for digesting and absorbing nutrients from meat and organs. The carnivore elimination protocol emphasizes skipping

#### Structuring the Agent's Output: Data Models

To make the agent's self-critique process reliable, we need to enforce a specific output structure. We use Pydantic `BaseModel` to define two data classes:

1.  `Reflection`: This class structures the self-critique, requiring the agent to identify what information is `missing` and what is `superfluous` (unnecessary).
2.  `AnswerQuestion`: This class structures the entire response. It forces the agent to provide its main `answer`, a `reflection` (using the `Reflection` class), and a list of `search_queries`.


In [11]:
class Reflection(BaseModel):
	missing: str = Field(description="What information is missing")
	superfluous: str = Field(description="What information is unnecessary")

class AnswerQuestion(BaseModel):
	answer: str = Field(description="Main response to the question")
	reflection: Reflection = Field(description="Self-critique of the answer")
	search_queries: List[str] = Field(description="Queries for additional research")

#### Binding Tools to the Responder

Now, we bind the `AnswerQuestion` data model as a **tool** to our LLM chain. This crucial step forces the LLM to generate its output in the exact JSON format defined by our Pydantic classes. The LLM doesn't just write text; it calls this "tool" to structure its entire thought process.

After invoking this new chain, we can see the structured output, including the initial answer, the self-critique, and the generated search queries:


In [73]:

import json
from IPython.display import display, JSON

initial_chain = first_responder_prompt| llm.bind_tools(tools=[AnswerQuestion])
response=initial_chain.invoke({"messages":[HumanMessage(question)]})
print("------question--------------")
print(question)
print("------first responder prompt--------------")
#print(first_responder_prompt)
print("------response.pretty_print()--------------")
response.pretty_print()
print("---Full Structured response JSON---")
display(JSON([response]))
print("---Full Structured response almost pretty print---")
print("--> response.content : ",response.content)
kwrgs = len(response.additional_kwargs)
print("--> response.additional_kwargs : ",kwrgs)
for i in response.additional_kwargs:
    print ("----> additional_kwargs : ",i)
print("--> response.response_metadata : ",response.response_metadata)
print("--> response.type : ",response.type)
print("--> response.name : ",response.name)
print("--> response.id : ",response.id)
print("--> response.example : ",response.example)
print("--> response.tool_calls : ",response.tool_calls)
print("--> response.invalid_tool_calls : ",response.invalid_tool_calls)
print("---End of Full Structured response almost pretty print---")


------question--------------
Any ideas for a healthy breakfast
------first responder prompt--------------
------response.pretty_print()--------------
================================== Ai Message ==================================
Tool Calls:
  AnswerQuestion (call_v8Y1TXniMMAjeo9IICP2bI8S)
 Call ID: call_v8Y1TXniMMAjeo9IICP2bI8S
  Args:
    answer: A truly healthy breakfast, grounded in human evolutionary biology, should prioritize nutrient-dense, bioavailable foods that support optimal physiology. Animal-based options, such as eggs, bone marrow, liver, and other organ meats, are ideal. These foods provide high-quality complete proteins, essential fatty acids, vitamins (A, D, B12), minerals (iron, zinc, selenium), and other vital nutrients in forms that are easily absorbed and used by the body.

Organ meats, particularly liver, are considered superfoods because they pack a concentrated punch of micronutrients that support energy, immunity, and cognitive function. Incorporating wild-ca

<IPython.core.display.JSON object>

---Full Structured response almost pretty print---
--> response.content :  
--> response.additional_kwargs :  2
----> additional_kwargs :  tool_calls
----> additional_kwargs :  refusal
--> response.response_metadata :  {'token_usage': {'completion_tokens': 401, 'prompt_tokens': 331, 'total_tokens': 732, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CJ99O7cCVHme5llrYEm6Yy2m2WxXo', 'finish_reason': 'tool_calls', 'logprobs': None}
--> response.type :  ai
--> response.name :  None
--> response.id :  run--832f3df6-48de-4c3f-8f5e-d99438bc6d90-0
--> response.example :  False
--> response.tool_calls :  [{'name': 'AnswerQuestion', 'args': {'answer': 'A truly healthy breakfast, grounded in human evolutionary biology, should prioritize nutrient-

In [74]:
answer_content = response.tool_calls[0]['args']['answer']
print("---Initial Answer---")
print(answer_content)

---Initial Answer---
A truly healthy breakfast, grounded in human evolutionary biology, should prioritize nutrient-dense, bioavailable foods that support optimal physiology. Animal-based options, such as eggs, bone marrow, liver, and other organ meats, are ideal. These foods provide high-quality complete proteins, essential fatty acids, vitamins (A, D, B12), minerals (iron, zinc, selenium), and other vital nutrients in forms that are easily absorbed and used by the body.

Organ meats, particularly liver, are considered superfoods because they pack a concentrated punch of micronutrients that support energy, immunity, and cognitive function. Incorporating wild-caught fish or pastured meats enhances nutrient density further. Unlike plants, animal tissues lack antinutrients like phytates, oxalates, and lectins that can impair mineral absorption, cause inflammation, or disrupt gut health.

Conventional dietary advice promoting carbohydrates and fiber from plants ignores the evolutionary evi

In [75]:
Reflection_content = response.tool_calls[0]['args']['reflection']
#Super_content = response.tool_calls[0]['args']['superfluous']
print("---Reflection Answer---")
print(Reflection_content)
#print(Super_content)

---Reflection Answer---
{'missing': "Specific scientific studies comparing nutrient bioavailability between animal and plant foods, and detailed analysis of plant antinutrients' effects on human health.", 'superfluous': 'Overly detailed explanations of plant defense compounds not directly relevant to structuring a practical breakfast suggestion.'}


In [76]:
search_queries = response.tool_calls[0]['args']['search_queries']
print("---Search Queries---")
print(search_queries)

---Search Queries---
['antinutrients in plants bioavailability', 'evolutionary nutrition and human diet', 'benefits of organ meats and animal foods']


### Tool Execution

Now that the Responder has generated search queries based on its self-critique, the next step is to actually *execute* those searches. We'll define a function, `execute_tools`, that takes the agent's state, extracts the search queries, runs them through the Tavily tool, and returns the results.

We will also manage the conversation history in `response_list`:


In [90]:
response_list=[]
response_list.append(HumanMessage(content=question))
response_list.append(response)
for i in range(0, len(response_list)):
    print('--- ',i,'  ---')
    print (response_list[i])
    print('--------------')

---  0   ---
content='Any ideas for a healthy breakfast' additional_kwargs={} response_metadata={}
--------------
---  1   ---
content='' additional_kwargs={'tool_calls': [{'id': 'call_v8Y1TXniMMAjeo9IICP2bI8S', 'function': {'arguments': '{"answer":"A truly healthy breakfast, grounded in human evolutionary biology, should prioritize nutrient-dense, bioavailable foods that support optimal physiology. Animal-based options, such as eggs, bone marrow, liver, and other organ meats, are ideal. These foods provide high-quality complete proteins, essential fatty acids, vitamins (A, D, B12), minerals (iron, zinc, selenium), and other vital nutrients in forms that are easily absorbed and used by the body.\\n\\nOrgan meats, particularly liver, are considered superfoods because they pack a concentrated punch of micronutrients that support energy, immunity, and cognitive function. Incorporating wild-caught fish or pastured meats enhances nutrient density further. Unlike plants, animal tissues lack 

In [91]:
tool_call=response.tool_calls[0]
search_queries = tool_call["args"].get("search_queries", [])
print(search_queries)

['antinutrients in plants bioavailability', 'evolutionary nutrition and human diet', 'benefits of organ meats and animal foods']


In [92]:
tavily_tool=TavilySearchResults(max_results=3)



def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    last_ai_message = state[-1]
    tool_messages = []
    for tool_call in last_ai_message.tool_calls:
        if tool_call["name"] in ["AnswerQuestion", "ReviseAnswer"]:
            call_id = tool_call["id"]
            search_queries = tool_call["args"].get("search_queries", [])
            query_results = {}
            for query in search_queries:
                result = tavily_tool.invoke(query)
                query_results[query] = result
            tool_messages.append(ToolMessage(
                content=json.dumps(query_results),
                tool_call_id=call_id)
            )
    return tool_messages

In [93]:
tool_response = execute_tools(response_list)
# Use .extend() to add all tool messages from the list
response_list.extend(tool_response)

In [94]:
tool_response

[ToolMessage(content='{"antinutrients in plants bioavailability": [{"title": "Plant food anti-nutritional factors and their reduction strategies", "url": "https://fppn.biomedcentral.com/articles/10.1186/s43014-020-0020-5", "content": "tannins, metal chelators, protease inhibitors, saponins, cyanogens, phytic acid, isoflavonoids, etc. (Pariza 1996. Toxic substances. Food chemistry (3rd ed. pp. 825\\u2013841). New York: Marcel Dekker.\\")). One of the major anti-nutrients is phytate, which chelates and mainly affects bioavailability of calcium and other micronutrients such as iron, copper and zinc. Some other factors such as polyphenols and oxalates are also considered as anti-nutrients that can limit food mineral bioavailability [...] Food Science & Technology, 37(7), 749\\u2013758.\\")). In addition to decreasing nutrient bioavailability, anti-nutrients can become toxic when present beyond a certain amount. Therefore, reduction in the levels of anti-nutritional factors in edible crops 

In [95]:
response_list

[HumanMessage(content='Any ideas for a healthy breakfast', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_v8Y1TXniMMAjeo9IICP2bI8S', 'function': {'arguments': '{"answer":"A truly healthy breakfast, grounded in human evolutionary biology, should prioritize nutrient-dense, bioavailable foods that support optimal physiology. Animal-based options, such as eggs, bone marrow, liver, and other organ meats, are ideal. These foods provide high-quality complete proteins, essential fatty acids, vitamins (A, D, B12), minerals (iron, zinc, selenium), and other vital nutrients in forms that are easily absorbed and used by the body.\\n\\nOrgan meats, particularly liver, are considered superfoods because they pack a concentrated punch of micronutrients that support energy, immunity, and cognitive function. Incorporating wild-caught fish or pastured meats enhances nutrient density further. Unlike plants, animal tissues lack antinutrien

### Defining the Revisor

The **Revisor** is the final piece of the Reflection loop. Its job is to take the original answer, the self-critique, and the new information from the tool search, and then generate an improved, more evidence-based response.

We create a new set of instructions (`revise_instructions`) that guide the Revisor. These instructions emphasize:
- Incorporating the critique.
- Adding numerical citations from the research.
- Distinguishing between correlation and causation.
- Adding a "References" section.


In [96]:
revise_instructions = """Revise your previous answer using the new information, applying the rigor and evidence-based approach of Dr. David Attia.
- Incorporate the previous critique to add clinically relevant information, focusing on mechanistic understanding and individual variability.
- You MUST include numerical citations referencing peer-reviewed research, randomized controlled trials, or meta-analyses to ensure medical accuracy.
- Distinguish between correlation and causation, and acknowledge limitations in current research.
- Address potential biomarker considerations (lipid panels, inflammatory markers, and so on) when relevant.
- Add a "References" section to the bottom of your answer (which does not count towards the word limit) in the form of:
- [1] https://example.com
- [2] https://example.com
- Use the previous critique to remove speculation and ensure claims are supported by high-quality evidence. Keep response under 250 words with precision over volume.
- When discussing nutritional interventions, consider metabolic flexibility, insulin sensitivity, and individual response variability.
"""
revisor_prompt = prompt_template.partial(first_instruction=revise_instructions)

#### Structuring the Revisor's Output

Just as we did with the Responder, we define a Pydantic class, `ReviseAnswer`, to structure the Revisor's output. This class inherits from `AnswerQuestion` but adds a new field for `references`, ensuring the agent includes citations in its revised answer.

We then bind this new tool to the revisor chain:


In [97]:
class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question."""
    references: List[str] = Field(description="Citations motivating your updated answer.")
revisor_chain = revisor_prompt | llm.bind_tools(tools=[ReviseAnswer])

#### Invoking the Revisor

Finally, we invoke the `revisor_chain`, passing it the entire conversation history: the original question, the first response (with its critique and search queries), and the new information gathered from the tool search. This provides the Revisor with all the context it needs to generate a final, improved answer.


In [98]:
response = revisor_chain.invoke({"messages": response_list})
print("---Revised Answer with References---")
print(response.tool_calls[0]['args'])

---Revised Answer with References---
{'answer': 'A high-quality, nutrient-dense breakfast rooted in evolutionary biology should focus on animal-based foods, particularly organ meats like liver and heart, eggs, and oxidative-rich seafood. These foods provide highly bioavailable vitamins (A, D, B12), essential minerals (iron, zinc, selenium), and complete proteins, crucial for energy, immunity, and metabolic health. Organ meats are especially nutrient-dense, often outperforming plant sources in bioavailability and micronutrient content (e.g., heme iron), and lack antinutrients like phytates and oxalates that impair mineral absorption and may cause toxicity.\n\nConventional wisdom emphasizing fiber for gut health does not align with human evolutionary evidence, which suggests humans thrived on animal foods with minimal plant fiber intake. Excess dietary fiber can disrupt gut microbiota and lead to dysbiosis (Ferrario et al., 2020). The carnivore protocol demonstrates that humans can thriv

In [99]:
response_list.append(response)

## Building the Graph

Now we will use **LangGraph** to assemble these components—Responder, Tool Executor, and Revisor—into a cohesive, cyclical workflow. A graph is a natural way to represent this process, where nodes represent the different stages of thinking and edges represent the flow of information between them.

### Defining the Event Loop

The core of our graph is the event loop. This function determines whether the agent should continue its revision process or if it has reached a satisfactory conclusion. We'll set a maximum number of iterations to prevent the agent from getting stuck in an infinite loop:


In [100]:
MAX_ITERATIONS = 4

In [101]:
def event_loop(state: List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations >= MAX_ITERATIONS:
        return END
    return "execute_tools"

In [102]:
graph=MessageGraph()

graph.add_node("respond", initial_chain)
graph.add_node("execute_tools", execute_tools)
graph.add_node("revisor", revisor_chain)

/tmp/ipykernel_301/2942375001.py:1: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph=MessageGraph()


In [103]:
graph.add_edge("respond", "execute_tools")
graph.add_edge("execute_tools", "revisor")

In [104]:
graph.add_conditional_edges("revisor", event_loop)
graph.set_entry_point("respond")

## Running the Agent

With our graph compiled, we're ready to run the full Reflection agent. We'll give it a new, more complex query that requires careful, evidence-based advice.

As the agent runs, we can see the entire process unfold: the initial draft, the self-critique, the tool searches, and the final, revised answer that incorporates the new evidence.


In [119]:
app = graph.compile()
responses = app.invoke(
    """Please advise should I buy or sell home in current situation"""
)
print(len(responses))
for i in range(0, len(responses)):
    print ('---  ',i,'  ---')
    print("--> response.type : ",responses[i].type)
    match responses[i].type:
        case "human":
            print("----> response.content : ", responses[i].content)
            responses[i].pretty_print()
        case "ai":
            responses[i].pretty_print()
        case "tool":
            responses[i].pretty_print()
    display(JSON([responses[i]]))

11
---   0   ---
--> response.type :  human
----> response.content :  Please advise should I buy or sell home in current situation
================================ Human Message =================================

Please advise should I buy or sell home in current situation


<IPython.core.display.JSON object>

---   1   ---
--> response.type :  ai
================================== Ai Message ==================================
Tool Calls:
  AnswerQuestion (call_jyr2vPoKZBgzLQ4WvxG4kGIE)
 Call ID: call_jyr2vPoKZBgzLQ4WvxG4kGIE
  Args:
    answer: While I am not a financial advisor, I can share insights on the benefits of animal-based nutrition. When considering lifestyle choices, focusing on nutrient-dense, bioavailable foods derived from animals—such as organ meats, eggs, and meat—provides essential vitamins and minerals in a form readily utilized by the human body. This is rooted in our evolutionary history, where early humans relied heavily on animal foods for survival and optimal health. In the context of investment or housing decisions, the principle of prioritizing highly bioavailable, nature-designed nutrients aligns with the idea that human health flourishes when supported by the foods our physiology has adapted to consume.

Contrasting this with the notion of relying heavily on proce

<IPython.core.display.JSON object>

---   2   ---
--> response.type :  tool
================================= Tool Message =================================

{}


...

<IPython.core.display.JSON object>

---   9   ---
--> response.type :  tool
================================= Tool Message =================================

{}


<IPython.core.display.JSON object>

---   10   ---
--> response.type :  ai
================================== Ai Message ==================================
Tool Calls:
  ReviseAnswer (call_tau2dmpBTJj1DRb2W3hmI6Z1)
 Call ID: call_tau2dmpBTJj1DRb2W3hmI6Z1
  Args:
    answer: Making a decision to buy or sell a home should primarily be based on market and personal financial factors. From a nutritional perspective, humans are evolutionarily designed to thrive on nutrient-dense, bioavailable foods from animals, such as organ meats, eggs, and meat. These foods supply essential vitamins, minerals, and amino acids in forms that are easily absorbed, supporting metabolic health and resilience. In contrast, plant foods contain antinutrients like oxalates, lectins, and phytates—defense chemicals that can interfere with nutrient absorption and promote inflammation when consumed excessively. Our ancestors' limited exposure to these compounds suggests that an animal-based diet aligns with our evolutionary needs, minimizing inflammatory

<IPython.core.display.JSON object>

In [120]:
print("--- Initial Draft Answer ---")
initial_answer = responses[1].tool_calls[0]['args']['answer']
print(initial_answer)
print("\n")

print("--- Intermediate and Final Revised Answers ---")
answers = []

# Loop through all messages in reverse to find all tool_calls with answers
for msg in reversed(responses):
    if getattr(msg, 'tool_calls', None):
        for tool_call in msg.tool_calls:
            answer = tool_call.get('args', {}).get('answer')
            if answer:
                answers.append(answer)

# Print all collected answers
for i, ans in enumerate(answers):
    label = "Final Revised Answer" if i == 0 else f"Intermediate Step {len(answers) - i}"
    print(f"{label}:\n{ans}\n")


--- Initial Draft Answer ---
While I am not a financial advisor, I can share insights on the benefits of animal-based nutrition. When considering lifestyle choices, focusing on nutrient-dense, bioavailable foods derived from animals—such as organ meats, eggs, and meat—provides essential vitamins and minerals in a form readily utilized by the human body. This is rooted in our evolutionary history, where early humans relied heavily on animal foods for survival and optimal health. In the context of investment or housing decisions, the principle of prioritizing highly bioavailable, nature-designed nutrients aligns with the idea that human health flourishes when supported by the foods our physiology has adapted to consume.

Contrasting this with the notion of relying heavily on processed plant foods, which contain anti-nutrients like oxalates, lectins, and phytates, raises questions about their long-term safety and impact. These plant defense compounds evolved to deter herbivores and are of

## Authors


[Joseph Santarcangelo](https://author.skills.network/instructors/joseph_santarcangelo)


[Faranak Heidari](https://author.skills.network/instructors/faranak_heidari)


### Other Contributors


[Abdul Fatir](https://author.skills.network/instructors/abdul_fatir)


## Change Log


<details>
    <summary>Click here for the changelog</summary>


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2025-06-24|0.5|Leah Hanson|QA review and grammar fixes|
|2025-06-24|0.4|Steve Ryan|ID review and format/typo fixes|
|2025-06-16|0.3|Abdul Fatir|Updated Lab|
|2025-06-10|0.2|Joseph Santarcangelo|Changed Project Architecture|
|2025-05-30|0.1|Faranak Heidari and Joseph Santarcangelo |Created Lab|

</details>


Copyright © IBM Corporation. All rights reserved.
